In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf

from sklearn.model_selection import train_test_split

import os
import io
import time

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import utils
import model_encdec


In [0]:
checkpoint_dir = '/content/drive/My Drive/IFT6759-PROJECT2/config5'

In [0]:
# path_source = ['/content/drive/My Drive/IFT6759-PROJECT2/data/split_train.lang1', '/content/drive/My Drive/IFT6759-PROJECT2/data/unaligned.fr']
# path_destination = ['/content/drive/My Drive/IFT6759-PROJECT2/data/split_train.lang2.txt', '/content/drive/My Drive/IFT6759-PROJECT2/data/unaligned.fr']

path_source = '/content/drive/My Drive/IFT6759-PROJECT2/data/split_train.lang1_95'
path_destination = '/content/drive/My Drive/IFT6759-PROJECT2/data/split_train.lang2_95'


# print(path_source[0])
# print(path_source[1])

# print(path_destination[0])
# print(path_destination[1])
# path_source = '/content/drive/My Drive/IFT6759-PROJECT2/data/train.lang1'
# path_destination = '/content/drive/My Drive/IFT6759-PROJECT2/data/train.lang2'

num_examples = 10450

input_tensor_train, target_tensor_train, inp_lang, targ_lang = utils.load_dataset(path_source, path_destination, num_examples)
max_length_targ, max_length_inp = utils.max_length(target_tensor_train), utils.max_length(input_tensor_train)

# input_tensor_train, target_tensor_train, inp_lang, targ_lang = utils.load_dataset_2_files(path_source, path_destination, num_examples, 3)
# max_length_targ, max_length_inp = utils.max_length(target_tensor_train), utils.max_length(input_tensor_train)


# max_length_targ, max_length_inp = utils.max_length(target_tensor), utils.max_length(input_tensor)
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, shuffle=False)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor_train, target_tensor_train, test_size=0.2, shuffle=True)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
val_dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

encoder = model_encdec.Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
attention_layer = model_encdec.BahdanauAttention(8)
decoder = model_encdec.Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')



In [0]:

"""## Checkpoints (Object-based saving)"""

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [12]:
input_tensor_train.shape

(8360, 94)

In [13]:
target_tensor_train.shape

(8360, 106)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

@tf.function
def val_step(inp, targ, enc_hidden):
  loss = 0

  enc_output, enc_hidden = encoder(inp, enc_hidden)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  for t in range(1, targ.shape[1]):
    # passing enc_output to the decoder
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

    loss += loss_function(targ[:, t], predictions)

    # using teacher forcing
    dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  return batch_loss


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = utils.preprocess_sentence(sentence)
  # print(sentence)

  # inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]

  inputs = []

  for i in sentence.split(' '):
    if inp_lang.word_index.get(i) != None:
      inputs.append(inp_lang.word_index.get(i))

  # print(inputs) 

  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()


    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot
    else:
      result += targ_lang.index_word[predicted_id] + ' '

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))


In [15]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [18]:
EPOCHS = 3

best_val_loss = np.inf

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
      print('Epoch {} Training Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
      

  print('Epoch {} Training Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = val_step(inp, targ, enc_hidden)
    total_loss += batch_loss
  total_loss_ = total_loss / steps_per_epoch
  print('Epoch {} Validation Loss {:.4f}'.format(epoch + 1,
                                      total_loss_))

  if best_val_loss > total_loss_:
    best_val_loss = total_loss_
    checkpoint.save(file_prefix = checkpoint_prefix)
    print("-- saving at best validation loss")


Epoch 1 Training Loss 0.0026
Time taken for 1 epoch 0.3492105007171631 sec

Epoch 1 Validation Loss 0.0033
-- saving at best validation loss
Epoch 2 Training Loss 0.0035
Time taken for 1 epoch 0.33395934104919434 sec

Epoch 2 Validation Loss 0.0036
Epoch 3 Training Loss 0.0029
Time taken for 1 epoch 0.33498334884643555 sec

Epoch 3 Validation Loss 0.0030
-- saving at best validation loss


In [0]:

# checkpoint.restore('/content/drive/My Drive/IFT6759-PROJECT2/synt_17600-4/synt_17600-4/ckpt-33')

translate(u'thank you.')

translate(u'i need to go the cinema')

translate(u'it is time to sleep')

translate(u'i would like to go to friends house')

translate(u'i would like to see him again')

Input: <start> thank you . <end>
Predicted translation: merci beaucoup . 
Input: <start> i need to go the cinema <end>
Predicted translation: je dois , le droit a la limitation de poste . 
Input: <start> it is time to sleep <end>
Predicted translation: il est temps au ski . 
Input: <start> i would like to go to friends house <end>
Predicted translation: je voudrais attirer par semaine . 
Input: <start> i would like to see him again <end>
Predicted translation: je voudrais que ca a nouveau . 


In [0]:
path_source_val = '/content/drive/My Drive/IFT6759-PROJECT2/data/split_val.lang1_05'
path_destination_val = '/content/drive/My Drive/IFT6759-PROJECT2/data/split_val.lang2_05'

validation_input = []
validation_results = []
validation_target = []

lines_source_val = io.open(path_source_val, encoding='UTF-8').read().strip().split('\n')
lines_destination_val = io.open(path_destination_val, encoding='UTF-8').read().strip().split('\n')

# result, sentence, attention_plot = evaluate(lines_source_val[6])

for index, l in enumerate(lines_source_val):
    result, sentence, attention_plot = evaluate(l)
    validation_results.append(result)
    validation_input.append(l)
    validation_target.append(lines_destination_val[index])

In [0]:
validation_results[:10]

['je suis , et je vais en un . ',
 'je suis d accord par plusieurs fois que je me suis implique la premiere fois que je me suis ici d un des plus interessants des femmes . ',
 'les depenses , le probleme de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question de la question ',
 'il est la tactique deployee par les etats membres de l aide soient grace aux citoyens . ',
 'quelque chose que j ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu , en ai vu 

In [0]:
validation_target[:10]

["Je suis Jon M. Chu , et je ne suis pas un danseur . Je ne suis pas un chorégraphe , en réalité je suis un producteur de films , un raconteur d' histoires .",
 "Je rejoins ce qu' ont dit quelques orateurs quant à la situation des femmes .",
 "À Téhéran , le cœur et l' âme politique du pays , seuls moins d' un tiers des électeurs se sont présentés aux urnes .",
 'La capacité des gouvernants des États en la matière est rendue optionnelle . Ils sont pourtant élus par le peuple , eux !',
 "Il s' est passé quelque chose quand j' ai commencé à voyager pour V - day il y a huit ans . Je me suis perdue .",
 "Ce qui se passe en Tunisie en ce moment pose question à chacun d' entre nous et pose question à l' Europe , avec tous ses retards et toutes ses erreurs .",
 "Il n' est pas transposable à volonté car chaque État membre présente un contexte qui lui est propre .",
 "J' ai ajouté une pièce à ma maison .",
 "par écrit . - Dans le contexte de la crise de l' euro , les États membres ne doivent pl

In [0]:
validation_input[:10]

["i 'm jon m. chu and i 'm not a dancer i 'm not a choreographer -- i 'm actually a filmmaker a storyteller",
 'i agree with what a few of the speakers said concerning the situation of women',
 "in tehran the country 's political heart and soul less than a third of the electorate turned out",
 'it makes the capacity of the leaders of the member states to act in this matter merely optional even though they are the ones elected by the people',
 'something happened when i began traveling in v - day eight years ago i got lost',
 'what is happening in tunisia at the moment poses questions for all of us and poses questions for europe with all its delays and mistakes',
 'it can not be copied arbitrarily since each member state has its own specific conditions',
 'i added a room to my house',
 'in the context of the euro crisis member states should no longer escape penalties if their deficits get out of control',
 'i said the same thing here in parliament two years ago and you voted to discharg

In [0]:
with open('validation_input.txt', 'w') as f:
    for item in validation_input:
        f.write("%s\n" % item)

In [0]:
with open('validation_results.txt', 'w') as f:
    for item in validation_results:
        f.write("%s\n" % item)

In [0]:
with open('validation_target.txt', 'w') as f:
    for item in validation_target:
        f.write("%s\n" % item)

In [0]:
import evaluator

In [0]:
!pip install sacrebleu


In [0]:
!python '/content/evaluator.py' --do-not-run-model --target-file-path='/content/validation_results.txt' --input-file-path='/content/drive/My Drive/IFT6759-PROJECT2/data/split_val.lang2_05'


['5.4', '4.4', '0.1', '2.3', '0.1', '0.7', '2.0', '0.0', '0.3', '0.2', '0.0', '0.6', '8.5', '0.1', '6.3', '7.3', '4.0', '4.8', '4.0', '0.1', '2.2', '0.1', '2.4', '0.2', '5.7', '15.1', '8.9', '2.4', '6.7', '2.3', '3.8', '2.3', '1.9', '2.0', '1.7', '12.4', '11.7', '4.8', '23.6', '2.2', '6.4', '5.0', '0.1', '0.0', '0.0', '8.1', '0.1', '8.9', '0.1', '3.7', '3.8', '4.2', '13.1', '5.1', '0.2', '0.1', '1.3', '4.0', '6.9', '4.5', '1.4', '9.5', '0.1', '2.7', '1.7', '2.0', '2.3', '6.3', '4.2', '0.3', '4.1', '1.8', '2.1', '0.1', '9.7', '4.5', '0.2', '0.0', '4.9', '13.5', '2.4', '3.0', '0.3', '23.6', '1.3', '14.5', '6.3', '0.8', '4.7', '0.0', '0.0', '0.6', '6.2', '7.4', '1.6', '0.7', '3.7', '0.2', '4.9', '0.1', '2.0', '0.2', '2.9', '1.4', '2.0', '3.8', '0.2', '2.7', '0.0', '0.1', '0.3', '2.9', '0.4', '1.3', '5.8', '2.7', '4.5', '2.9', '4.3', '0.4', '2.7', '0.1', '0.1', '0.1', '4.1', '0.0', '0.0', '2.0', '0.5', '4.8', '1.8', '1.8', '1.8', '4.6', '3.8', '0.2', '0.1', '2.4', '0.4', '2.3', '2.4', '1.2